In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [1]:
# Step 1: Import Libraries
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain


In [3]:
# Step 2: Introduction Prompt
introduction_template = "You are a sentiment classifier. Determine if the input is Positive, Neutral, or Negative."
introduction_prompt = PromptTemplate.from_template(introduction_template)


In [4]:
# Step 3: Few-Shot Examples Prompt
examples = [
    {"text": "The pizza was delicious and perfectly baked!", "label": "Positive"},
    {"text": "The food was okay, nothing special.", "label": "Neutral"},
    {"text": "The service was terrible and I had to wait an hour.", "label": "Negative"}
]

example_template = PromptTemplate.from_template("Input: {text}\nSentiment: {label}\n")

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="Here are some examples:",
    suffix="",
    input_variables=[]
)


In [5]:
# Step 4: Execution Prompt
execution_template = "Now analyze this review:\nInput: {input_text}\nSentiment:"
execution_prompt = PromptTemplate.from_template(execution_template)


In [6]:
# Step 5: Combine using PipelinePromptTemplate
final_template = """{intro}

{examples}

{execution}"""

final_prompt = PromptTemplate.from_template(final_template)

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=final_prompt,
    pipeline_prompts=[
        ("intro", introduction_prompt),
        ("examples", few_shot_prompt),
        ("execution", execution_prompt)
    ]
)


/var/folders/g1/5_fr35dx4bg267_nq27dz3tw0000gn/T/ipykernel_80514/1046937676.py:10: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


In [8]:
# Step 6: Create Chain and Run
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = LLMChain(llm=llm, prompt=pipeline_prompt)

# Run a test input
input_text = "The ambiance was great, but the food was underwhelming."
result = chain.invoke({"input_text": input_text})

print("LLM Output:\n", result["text"])


LLM Output:
 Sentiment: Neutral
